In [ ]:
# 사용자 현재 위치부터 반경 5km이내 공원 가져오는 SELECT문 (python코드로 걸러내는 것 보다 이게 빠름)

import pymysql
from dotenv import load_dotenv
import os 

load_dotenv()

# 팀원 계정 정보
conn = pymysql.connect(
    host=os.getenv("DB_HOST"),
    port=int(os.getenv("DB_PORT")),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    db=os.getenv("DB_NAME"),
    charset=os.getenv("DB_CHARSET", "utf8mb4")
)

center_lat, center_lon = 37.534141, 126.968200 # 사용자한테 받은 위도, 경도

cur = conn.cursor(pymysql.cursors.DictCursor)

query = """
SELECT ID, Latitude, Longitude,
       (6371 * ACOS(
           COS(RADIANS(%s)) * COS(RADIANS(latitude)) *
           COS(RADIANS(longitude) - RADIANS(%s)) +
           SIN(RADIANS(%s)) * SIN(RADIANS(latitude))
       )) AS distance
FROM tb_parks
HAVING distance <= 5
ORDER BY distance;
"""

cur.execute(query, (center_lat, center_lon, center_lat))
parks_list = cur.fetchall()

cur.close()
conn.close()

In [ ]:
len(parks_list)

12

In [ ]:
# 반경 5km 떨어진 공원 리스트에 있는 공원 점수 가져오기

import pymysql
from dotenv import load_dotenv
import os 

load_dotenv()

# 팀원 계정 정보
conn = pymysql.connect(
    host=os.getenv("DB_HOST"),
    port=int(os.getenv("DB_PORT")),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    db=os.getenv("DB_NAME"),
    charset=os.getenv("DB_CHARSET", "utf8mb4")
)

cur = conn.cursor(pymysql.cursors.DictCursor)

parks_score_list = []  # 여러 공원 점수 저장할 리스트

for park in parks_list:
    ID = park['ID']
    
    query = """
    SELECT *
    FROM tb_parks_score
    WHERE ParkID = %s;
    """
    
    cur.execute(query, (ID,))
    rows = cur.fetchall()
    
    # 결과 누적
    parks_score_list.extend(rows)

cur.close()
conn.close()

In [ ]:
parks_score_list[0]

{'ID': 9,
 'ParkID': 9,
 'Name': '용산가족공원',
 'Nature': 0.505,
 'Convenience': 0.526,
 'Safety': 0.679,
 'Activity': 0.697,
 'Social': 0.0,
 'Trust': 0.827}

In [ ]:
from typing import Dict, Any, List

# ===================== 감정 가중치(연구 기반) =====================

def get_emotion_base_weights() -> Dict[str, Dict[str, float]]:
    # 5지표 합=1 (정규화)
    return {
        "불안":   {"Nature": 0.20, "Convenience": 0.25, "Safety": 0.40, "Activity": 0.05, "Social": 0.10},
        "우울":   {"Nature": 0.45, "Convenience": 0.25, "Safety": 0.20, "Activity": 0.10, "Social": 0.00},
        "스트레스":{"Nature": 0.30, "Convenience": 0.15, "Safety": 0.05, "Activity": 0.50, "Social": 0.00},
        "행복":   {"Nature": 0.25, "Convenience": 0.15, "Safety": 0.05, "Activity": 0.30, "Social": 0.25},
        "에너지": {"Nature": 0.35, "Convenience": 0.20, "Safety": 0.10, "Activity": 0.30, "Social": 0.05},
        "성취감": {"Nature": 0.20, "Convenience": 0.25, "Safety": 0.15, "Activity": 0.35, "Social": 0.05},
    }

def blend_emotion_weights(emotion_levels: Dict[str, int]) -> Dict[str, float]:
    """
    저장된 5지표(Nature/Convenience/Safety/Activity/Social)에 적용할 통합 가중치(합=1) 생성.
    emotion_levels: {"우울":1~5, "불안":1~5, ...} (0은 미고려)
    """
    base = get_emotion_base_weights()
    dims = ['Nature', 'Convenience', 'Safety', 'Activity', 'Social']

    levels = {k: int(emotion_levels.get(k, 0)) for k in base.keys()}
    active = {k: v for k, v in levels.items() if v > 0}
    if not active:
        return {d: (1.0/len(dims)) for d in dims}

    total = sum(active.values())
    agg = {d: 0.0 for d in dims}
    for emo, lv in active.items():
        contrib = lv / total
        for d in dims:
            agg[d] += base[emo][d] * contrib

    s = sum(agg.values())
    return {d: (agg[d]/s if s>0 else 0.0) for d in dims}

# ===================== 점수화된 공원 추천 =====================

def score_with_stored_indicators(park: Dict[str, Any], weights: Dict[str, float]) -> Dict[str, float]:
    """
    저장된 지표값 × 통합 가중치 → raw, final(신뢰도 있으면 곱)
    park 예: {"Name":"효창<시공원>", "Nature":0.446, "Convenience":0.462, "Safety":0.677, "Activity":1.0, "Social":0.15, "Trust":0.72}
    """
    dims = ['Name', 'Nature', 'Convenience', 'Safety', 'Activity', 'Social']
    raw = 0.0
    wsum = 0.0
    for d in dims:
        v = park.get(d)
        if isinstance(v, (int,float)):
            raw += float(v) * weights.get(d, 0.0)
            wsum += weights.get(d, 0.0)
    raw = (raw/wsum) if wsum > 0 else None

    Trust = park.get("Trust", None)
    if raw is None:
        final = None
    else:
        final = raw * float(Trust) if isinstance(Trust, (int,float)) else raw

    return {
        "raw_score": None if raw is None else round(raw, 3),
        'Trust': None if raw is None else round(Trust, 3),
        "final_score": None if final is None else round(final, 3)
    }

def recommend_from_scored_parks(parks_scored: List[Dict[str, Any]],
                                emotion_levels: Dict[str, int],
                                top_n: int = 5,
                                return_weights: bool = True) -> Dict[str, Any]:
    """
    이미 점수화된 공원 리스트를 받아 최종 추천 생성.
    parks_scored: [{"Name":..., "Nature":..., "Convenience":..., "Safety":..., "Activity":..., "Social":..., "Trust":(옵션)}]
    """
    weights = blend_emotion_weights(emotion_levels)
    results = []
    for p in parks_scored:
        s = score_with_stored_indicators(p, weights)
        results.append({
            "ID": p['ParkID'], # 공원 ID
            # "Name": p.get("Name",""), # 이름
            "raw_score": s["raw_score"], # 원점수
            "Trust": s["Trust"], # 신뢰도(0/NaN 구별)
            "final_score": s["final_score"], # 최종 점수
        })
    # 점수 높은 순으로 정리
    ranked = sorted(results, key=lambda x: (x["final_score"] is not None, x["final_score"]), reverse=True)
    final_result = ranked[:top_n]
    return final_result

# ===================== 데모 =====================
if __name__ == "__main__":
    """
    parks_scored 형식 (JSON)
    {'ID': 9, 'ParkID': 9, 'Name': '용산가족공원', 'Nature': 0.505, 'Convenience': 0.526,
    'Safety': 0.679, 'Activity': 0.697, 'Social': 0.0, 'Trust': 0.827}
    """
    parks_scored = parks_score_list
    emotions = {"우울":5, "불안":5, "스트레스":5, "행복":5, "에너지":1, "성취감":1}
    res = recommend_from_scored_parks(parks_scored, emotions, top_n=6)
res


[{'ID': 13, 'raw_score': 0.652, 'Trust': 0.945, 'final_score': 0.616},
 {'ID': 7, 'raw_score': 0.506, 'Trust': 0.913, 'final_score': 0.462},
 {'ID': 9, 'raw_score': 0.544, 'Trust': 0.827, 'final_score': 0.45},
 {'ID': 33, 'raw_score': 0.485, 'Trust': 0.885, 'final_score': 0.429},
 {'ID': 27, 'raw_score': 0.566, 'Trust': 0.737, 'final_score': 0.417},
 {'ID': 31, 'raw_score': 0.453, 'Trust': 0.885, 'final_score': 0.401}]